<a href="https://colab.research.google.com/github/visith1577/tensorflow_advanced/blob/main/autograph_horse_OR_human.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf
import tensorflow_datasets as tfds
import tensorflow_hub as hub
import matplotlib.pyplot as plt


from __future__ import absolute_import, division, print_function, unicode_literals
import numpy as np

In [2]:
splits, info = tfds.load('horses_or_humans', as_supervised=True, with_info=True, split=['train[:80%]', 'train[80%:]', 'test'], data_dir='./data', download=True)

(train_examples, validation_examples, test_examples) = splits

num_examples = info.splits['train'].num_examples
num_classes = info.features['label'].num_classes

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

0 examples [00:00, ? examples/s]

Shuffling and writing examples to ./data/horses_or_humans/3.0.0.incompleteTSZJEG/horses_or_humans-train.tfrecord


  0%|          | 0/1027 [00:00<?, ? examples/s]

0 examples [00:00, ? examples/s]

Shuffling and writing examples to ./data/horses_or_humans/3.0.0.incompleteTSZJEG/horses_or_humans-test.tfrecord


  0%|          | 0/256 [00:00<?, ? examples/s]

Dataset horses_or_humans downloaded and prepared to ./data/horses_or_humans/3.0.0. Subsequent calls will reuse this data.


In [3]:
BATCH_SIZE = 64
IMAGE_SIZE = 224

In [6]:
@tf.function
def map_fn(img, label):
  image_height=224
  image_width=224

  img=tf.image.resize(img, (image_height, image_width))
  img /= 255.0
  return img, label

In [7]:

test_image, test_label = list(train_examples)[0]

test_result = map_fn(test_image, test_label)

print(test_result[0].shape)
print(test_result[1].shape)

del test_image, test_label, test_result

(224, 224, 3)
()


In [8]:
def prepare_dataset(train_examples, validation_examples, test_examples, num_examples, map_fn, batch_size):
    
    train_ds = train_examples.map(map_fn).shuffle(buffer_size = num_examples).batch(batch_size) 
    
    valid_ds = validation_examples.map(map_fn).batch(batch_size)
    test_ds = test_examples.map(map_fn).batch(batch_size)
    
    return train_ds, valid_ds, test_ds

In [9]:
train_ds, valid_ds, test_ds = prepare_dataset(train_examples, validation_examples, test_examples, num_examples, map_fn, BATCH_SIZE)

In [12]:
MODULE_HANDLE = 'https://tfhub.dev/tensorflow/resnet_50/classification/1'
model = tf.keras.Sequential([
    hub.KerasLayer(MODULE_HANDLE, input_shape=(IMAGE_SIZE, IMAGE_SIZE, 3)),
    tf.keras.layers.Dense(num_classes, activation='softmax')
])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
keras_layer (KerasLayer)     (None, 1001)              25612201  
_________________________________________________________________
dense (Dense)                (None, 2)                 2004      
Total params: 25,614,205
Trainable params: 2,004
Non-trainable params: 25,612,201
_________________________________________________________________


In [11]:
def set_adam_optimizer():
    
    optimizer = tf.keras.optimizers.Adam() 
    
    return optimizer

In [13]:
def set_sparse_cat_crossentropy_loss():
    
    train_loss = tf.keras.losses.SparseCategoricalCrossentropy()  
    val_loss = tf.keras.losses.SparseCategoricalCrossentropy() 
   
    return train_loss, val_loss

In [15]:
def set_sparse_cat_crossentropy_accuracy():

    train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy() 
    val_accuracy = tf.keras.metrics.SparseCategoricalAccuracy() 
    
    return train_accuracy, val_accuracy

In [16]:
optimizer = set_adam_optimizer()
train_loss, val_loss = set_sparse_cat_crossentropy_loss()
train_accuracy, val_accuracy = set_sparse_cat_crossentropy_accuracy()

In [17]:
device = '/gpu:0' if tf.test.is_gpu_available() else '/cpu:0'
EPOCHS = 2

# Custom training step
def train_one_step(model, optimizer, x, y, train_loss, train_accuracy):
    '''
    Trains on a batch of images for one step.
    
    Args:
        model (keras Model) -- image classifier
        optimizer (keras Optimizer) -- optimizer to use during training
        x (Tensor) -- training images
        y (Tensor) -- training labels
        train_loss (keras Loss) -- loss object for training
        train_accuracy (keras Metric) -- accuracy metric for training
    '''
    with tf.GradientTape() as tape:
    
        predictions = model(x) 
        
        loss = train_loss(y, predictions)

    grads = tape.gradient(loss, model.trainable_variables) 
    
    optimizer.apply_gradients(zip(grads, model.trainable_variables)) 

    train_accuracy(y, predictions)
    
    return loss

Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.


Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.


In [18]:
def base_model():
    inputs = tf.keras.layers.Input(shape=(2))
    x = tf.keras.layers.Dense(64, activation='relu')(inputs)
    outputs = tf.keras.layers.Dense(1, activation='sigmoid')(x)
    model = tf.keras.Model(inputs=inputs, outputs=outputs)
    return model

test_model = base_model()

test_optimizer = set_adam_optimizer()
test_image = tf.ones((2,2))
test_label = tf.ones((1,))
test_train_loss, _ = set_sparse_cat_crossentropy_loss()
test_train_accuracy, _ = set_sparse_cat_crossentropy_accuracy()

test_result = train_one_step(test_model, test_optimizer, test_image, test_label, test_train_loss, test_train_accuracy)
print(test_result)

del test_result, test_model, test_optimizer, test_image, test_label, test_train_loss, test_train_accuracy

tf.Tensor(0.6931472, shape=(), dtype=float32)


In [19]:

# Decorate this function with tf.function to enable autograph on the training loop
@tf.function
def train(model, optimizer, epochs, device, train_ds, train_loss, train_accuracy, valid_ds, val_loss, val_accuracy):
    '''
    Performs the entire training loop. Prints the loss and accuracy per step and epoch.
    
    Args:
        model (keras Model) -- image classifier
        optimizer (keras Optimizer) -- optimizer to use during training
        epochs (int) -- number of epochs
        train_ds (tf Dataset) -- the train set containing image-label pairs
        train_loss (keras Loss) -- loss function for training
        train_accuracy (keras Metric) -- accuracy metric for training
        valid_ds (Tensor) -- the val set containing image-label pairs
        val_loss (keras Loss) -- loss object for validation
        val_accuracy (keras Metric) -- accuracy metric for validation
    '''
    step = 0
    loss = 0.0
    for epoch in range(epochs):
        for x, y in train_ds:
            # training step number increments at each iteration
            step += 1
            with tf.device(device_name=device):
                ### START CODE HERE ###
                # Run one training step by passing appropriate model parameters
                # required by the function and finally get the loss to report the results
                loss = train_one_step(model, optimizer, x, y, train_loss, train_accuracy)
                ### END CODE HERE ###
            # Use tf.print to report your results.
            # Print the training step number, loss and accuracy
            tf.print('Step', step, 
                   ': train loss', loss, 
                   '; train accuracy', train_accuracy.result())

        with tf.device(device_name=device):
            for x, y in valid_ds:
                # Call the model on the batches of inputs x and get the predictions
                y_pred = model(x)
                loss = val_loss(y, y_pred)
                val_accuracy(y, y_pred)
        
        # Print the validation loss and accuracy
        ### START CODE HERE ###
        tf.print('val loss', loss, '; val accuracy', val_accuracy.result())
        ### END CODE HERE ###

In [ ]:
train(model, optimizer, EPOCHS, device, train_ds, train_loss, train_accuracy, valid_ds, val_loss, val_accuracy)

Step 1 : train loss 0.687515736 ; train accuracy 0.6875
Step 2 : train loss 0.688142 ; train accuracy 0.6484375
Step 3 : train loss 0.686270535 ; train accuracy 0.677083313
Step 4 : train loss 0.684403181 ; train accuracy 0.69140625
Step 5 : train loss 0.688009858 ; train accuracy 0.68125
Step 6 : train loss 0.686363578 ; train accuracy 0.677083313
Step 7 : train loss 0.679627419 ; train accuracy 0.694196403
Step 8 : train loss 0.68272078 ; train accuracy 0.705078125
Step 9 : train loss 0.682456255 ; train accuracy 0.704861104
Step 10 : train loss 0.678223908 ; train accuracy 0.715625
Step 11 : train loss 0.678811133 ; train accuracy 0.727272749
Step 12 : train loss 0.677513242 ; train accuracy 0.735677063
Step 13 : train loss 0.678927481 ; train accuracy 0.7408759
val loss 0.683853567 ; val accuracy 0.697560966
Step 14 : train loss 0.679097414 ; train accuracy 0.740406334
Step 15 : train loss 0.676322937 ; train accuracy 0.747368395
Step 16 : train loss 0.677703619 ; train accuracy 0.

In [ ]:
test_imgs = []
test_labels = []

predictions = []
with tf.device(device_name=device):
    for images, labels in test_ds:
        preds = model(images)
        preds = preds.numpy()
        predictions.extend(preds)

        test_imgs.extend(images.numpy())
        test_labels.extend(labels.numpy())

In [ ]:
class_names = ['horse', 'human']

def plot_image(i, predictions_array, true_label, img):
    predictions_array, true_label, img = predictions_array[i], true_label[i], img[i]
    plt.grid(False)
    plt.xticks([])
    plt.yticks([])

    img = np.squeeze(img)

    plt.imshow(img, cmap=plt.cm.binary)

    predicted_label = np.argmax(predictions_array)
    
    # green-colored annotations will mark correct predictions. red otherwise.
    if predicted_label == true_label:
        color = 'green'
    else:
        color = 'red'
    
    # print the true label first
    print(true_label)
  
    # show the image and overlay the prediction
    plt.xlabel("{} {:2.0f}% ({})".format(class_names[predicted_label],
                                100*np.max(predictions_array),
                                class_names[true_label]),
                                color=color)

In [ ]:
index = 8 
plt.figure(figsize=(6,3))
plt.subplot(1,2,1)
plot_image(index, predictions, test_labels, test_imgs)
plt.show()